#### Whoops wrong update done

In [ ]:
%%pyspark
##Whoops Data was changed accidentaly

from delta.tables import *
from pyspark.sql.functions import *


deltaTable = DeltaTable.forPath(spark, "abfss://fabricsynapse12@msit-onelake.dfs.fabric.microsoft.com/SQLDB_Synapse.Lakehouse/Tables/productdescription")
 

# Update the table (reduce price of accessories by 10%)
deltaTable.update(
    condition = "ModifiedDate == '2013-04-30 00:00:00.000'",
    set = { "ModifiedDate": "'2021-04-30 00:00:00.000'" }
                )

#### Reading the deltatable history to find the change

In [2]:
%%pyspark
##Which Version was?
#//listingversions - Find the version
from delta.tables import *
deltaTable = DeltaTable.forPath(spark, "abfss://NextSynapse12@msit-onelake.dfs.fabric.microsoft.com/SynapseFabric.Datawarehouse/Tables/dbo/FactInternetSales_clone_retorepoint")
latestHistory = deltaTable.history(); 
latestHistory.show(100) 

##C:\Users\lilem\OneLake - Microsoft\NextSynapse12\SynapseFabric.Datawarehouse



StatementMeta(, f4b1170d-f735-406a-9af2-2b6c8084af1f, 4, Finished, Available)

+-------+--------------------+------+--------+---------+-------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+----------+
|version|           timestamp|userId|userName|operation|operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics|userMetadata|engineInfo|
+-------+--------------------+------+--------+---------+-------------------+----+--------+---------+-----------+--------------+-------------+----------------+------------+----------+
|      1|2023-09-14 16:58:...|  null|    null|     null|               null|null|    null|     null|       null|          null|         null|            null|        null|      null|
|      0|2023-09-14 16:58:...|  null|    null|     null|               null|null|    null|     null|       null|          null|         null|            null|        null|      null|
+-------+--------------------+------+--------+---------+-------------------+----+----

In [1]:
1+1 

StatementMeta(, , , SessionError, )

In [ ]:
%%pyspark
##Which Version was?
#//listingversions - Find the version
from delta.tables import *
deltaTable = DeltaTable.forPath(spark, "abfss://fabricsynapse12@msit-onelake.dfs.fabric.microsoft.com/SQLDB_Synapse.Lakehouse/Tables/productdescription")
latestHistory = deltaTable.history(); 
latestHistory.show(100) 

#### Here we have the many rows that change and also prior the change. 
( it should return 636 rows)
#### Once we confirm if we should restore to the previous state, I can overwrite the table results with the timetravel filter

In [ ]:
%%spark

//Timetravel. Get back on time. 
//Scala
val df_read_timetravel = spark.read.format("delta").option("timestampAsOf", "2023-06-14 14:25").load("abfss://fabricsynapse12@msit-onelake.dfs.fabric.microsoft.com/SQLDB_Synapse.Lakehouse/Tables/productdescription")
df_read_timetravel.createOrReplaceTempView("view_timetravel")
val view_timetravel = spark.sqlContext.sql ("select count(*)  from view_timetravel WHERE ModifiedDate = '2013-04-30 00:00:00.000'");
view_timetravel.show()

val df_read_current = spark.read.format("delta").load("abfss://fabricsynapse12@msit-onelake.dfs.fabric.microsoft.com/SQLDB_Synapse.Lakehouse/Tables/productdescription")
df_read_current.createOrReplaceTempView("view_current")
val df_current = spark.sqlContext.sql ("select count(*) from view_current WHERE ModifiedDate = '2021-04-30 00:00:00.000'");
df_current.show()

//save the files from a point in time.
df_read_timetravel.write.mode("overwrite").format("delta").save("abfss://fabricsynapse12@msit-onelake.dfs.fabric.microsoft.com/SQLDB_Synapse.Lakehouse/Tables/productdescription")

#### Confirming the status of the actual data. Once it was overwriten the information with the incorrect filter should not be there anymore
it should return 0

In [7]:
%%spark

//Timetravel. Get back on time. 
//Scala
val df_read_current = spark.read.format("delta").load("abfss://fabricsynapse12@msit-onelake.dfs.fabric.microsoft.com/SQLDB_Synapse.Lakehouse/Tables/productdescription")
df_read_current.createOrReplaceTempView("view_current")
val df_current = spark.sqlContext.sql ("select count(*) from view_current WHERE ModifiedDate = '2021-04-30 00:00:00.000'");
df_current.show()


StatementMeta(, 998373d0-07c1-481b-8b2c-4e4ccb0eb716, 10, Finished, Available)

+--------+
|count(1)|
+--------+
|     636|
+--------+

+--------+
|count(1)|
+--------+
|       0|
+--------+

df_read_timetravel: org.apache.spark.sql.DataFrame = [ProductDescriptionID: int, Description: string ... 2 more fields]
view_timetravel: org.apache.spark.sql.DataFrame = [count(1): bigint]
df_read_current: org.apache.spark.sql.DataFrame = [ProductDescriptionID: int, Description: string ... 2 more fields]
df_current: org.apache.spark.sql.DataFrame = [count(1): bigint]
